In [1]:
from ovcg import *
from pyccx.server import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper

In [2]:
ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)

use_relax = True

# tolerance setting
tol = 1e-4
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 2
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid/fluidN.cmd')
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid/dummyN')
fluid_ofile = VtkFile('fluid/resFNSD.pvd')
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid driver
ccx = CCXServer()
config = CouplingControl()
config.addr = ccx.addr  # get IP and port
config.create_coupling_step(
    1,
    study='CHT',
    nset='Inodes',
    elset='Interface',
    client2server=DFLUX_TAG,
    server2client=TEMP_TAG
)
# write coupling control file
config.write('couplingFNSD.ini')

# invoke our ccx_client solver
ccx.run('solid/solidD', ini_control='couplingFNSD.ini')

# accept solid connection request
ccx.accept_connection()

# initialize the coupling step, since there is only one step
ccx.initialize_coupling_step(server2client=TEMP_TAG, client2server=DFLUX_TAG)

scents = ccx.imesh.centres
snodes = ccx.imesh.coordinates

In [5]:
Tf = 'Tf'
Ts = 'Ts'
Ff = 'Ff'
Fs = 'Fs'
Ff2Fs = {Tf:Ts}
Ts2Tf = {Fs:Ff}
f2s_mapper, s2f_mapper = generate_mapper(fnodes, [snodes,scents], Ff2Fs, Ts2Tf,r1=0.05,r2=0.2)

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=fnodes.shape[0], value=0.0)
fluxS = InterfaceData(size=scents.shape[0], value=0.0)
tempF = InterfaceData(size=fnodes.shape[0], value=1000.0)
tempS = InterfaceData(size=snodes.shape[0], value=600.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 100
avg_pc_iterations = 0.0
flog = open('FNSD.log', mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting\n')
flog.close()
flog = open('FNSD.log', mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    ccx.backup_state()
    
    # mapping
    s2f_mapper.put(Fs, fluxS.curr)
    s2f_mapper.apply(Fs, Ff)
    fluxF.curr[:] = s2f_mapper.get(Ff)
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=fluxF.curr
    )
    
    while True:
        # back up previous interface value
        fluxF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        f2s_mapper.put(Tf, tempF.curr)
        f2s_mapper.apply(Tf, Ts)
        tempS.curr[:] = f2s_mapper.get(Ts)
        # update solid temperature
        ccx.send_data_fields(temp=tempS.curr)
        
        # advance solid
        ccx.increment(dt)
        # update information from solid solver
        ccx.recv_info()
        # retrieve solid interface heat flux
        fluxS.curr[:] = ccx.get_data_fields()[0]
        
        # mapping
        s2f_mapper.put(Fs, fluxS.curr)
        s2f_mapper.apply(Fs, Ff)
        fluxF.curr[:] = s2f_mapper.get(Ff)
        
        # update residual
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            # notice the solid side to advance
            ccx.advance()
            if step == Nmax-1:
                # notice the solid side that the step has done
                ccx.finalize_coupling_step()
            else:
                ccx.continue_coupling_step()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(fluxF, step, pc_counts)
                under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=fluxF.curr
            )
            solverF.restore_previous_solutions()
            ccx.restore_state()
            # also notice the solid side that the current step is still ongoing
            ccx.continue_coupling_step()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

step=0, pc_iterations=30.
step=1, pc_iterations=15.
step=2, pc_iterations=13.
step=3, pc_iterations=12.
step=4, pc_iterations=12.
step=5, pc_iterations=12.
step=6, pc_iterations=11.
step=7, pc_iterations=11.
step=8, pc_iterations=11.
step=9, pc_iterations=12.
step=10, pc_iterations=12.
step=11, pc_iterations=12.
step=12, pc_iterations=12.
step=13, pc_iterations=12.
step=14, pc_iterations=10.
step=15, pc_iterations=11.
step=16, pc_iterations=10.
step=17, pc_iterations=10.
step=18, pc_iterations=10.
step=19, pc_iterations=9.
step=20, pc_iterations=10.
step=21, pc_iterations=10.
step=22, pc_iterations=9.
step=23, pc_iterations=9.
step=24, pc_iterations=9.
step=25, pc_iterations=9.
step=26, pc_iterations=8.
step=27, pc_iterations=8.
step=28, pc_iterations=7.
step=29, pc_iterations=8.
step=30, pc_iterations=7.
step=31, pc_iterations=6.
step=32, pc_iterations=7.
step=33, pc_iterations=6.
step=34, pc_iterations=6.
step=35, pc_iterations=6.
step=36, pc_iterations=6.
step=37, pc_iterations=6.
s

In [9]:
ccx.safe_terminate() # shut down the manager